# Abbreviation Disambiguation in Medical Texts - Data Modeling

This Notebook is in continuation of the notebook- 'Step 2- Data Preprocessing' and lists down:

1. Modeling Preprocessed data using: GridSearchCV on Logistic Regression, SVM and XG Boost.
2. Testing the models using Test set.
3. Comparing the models and identifying the Next Steps

## Step# 1: Loading Dataset

In [ ]:
#Importing the Required Python Packages
import string
import pandas as pd
import ast
from gensim.models.doc2vec import TaggedDocument
import spacy

from sklearn.preprocessing import LabelEncoder

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report
import numpy as np

pd.set_option('display.max_colwidth', None)

In [2]:
# Lets load the default english model of spacy
# !python -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm')

datapath = "../input/medal-emnlp/pretrain_subset"
# datapath = "archive/pretrain_subset"

# Lets load the train dataset.

train = pd.read_csv(datapath + '/train.csv')
# train = train[:100000]

# Lets load validation and test datasets as well
valid = pd.read_csv(f'{datapath}/valid.csv')
# test = pd.read_csv(f'{datapath}/test.csv')
# valid = valid[:10000]
# test = test[:1000]

In [4]:
# Lets create a function to create a new feature 'ABV' from dataset
def createFeature(df):    
    return [x.split(' ')[y] for x,y in zip(df['TEXT'], df['LOCATION'])]

train['ABV'] = createFeature(train)
valid['ABV'] = createFeature(valid)
# test['ABV'] = createFeature(test)

grouped = train.groupby(by=['ABV', 'LABEL'], as_index = False, sort = False).count()
grouped = grouped.sort_values(by='TEXT', ascending = False)

# topAbv = grouped['ABV'][:20]

train = train[train['ABV'].isin(topAbv)]
valid = valid[valid['ABV'].isin(topAbv)]
# test = test[test['ABV'].isin(topAbv)]

# Lets create a function to remove all the Punctuations from Text
def removePunctuation(df):
    return [t.translate(str.maketrans('','',string.punctuation)) for t in df['TEXT']]

# Lets create a function to Tokenize the Text column of dataset
def createTokens(df):
    return df['TEXT'].apply(lambda x: x.split(' '))

#Lets create a function to drop "Abstract_id", "Location" and "TEXT" columns from dataset
def dropCols(df):
    return df.drop(columns=['ABSTRACT_ID', 'LOCATION', 'TEXT'])

# Lets create a function to remove stop words from the Text column
def removeStop(df):
    stopWords = spacy.lang.en.stop_words.STOP_WORDS
    # Remove any stopwords which appear to be an Abbreviation
    [stopWords.remove(t) for t in df['ABV'].str.lower() if t in stopWords]
    return df['TOKEN'].apply(lambda x: [item for item in x if not item in stopWords])

def tolower(df):
    return [t.lower() for t in df['TEXT']]

def preProcessData(df):   
    df['TEXT'] = tolower(df)
    df['TEXT'] = removePunctuation(df)
    df['TOKEN'] = createTokens(df)
    df = dropCols(df)
    df['TOKEN'] = removeStop(df)
    return df

# Lets load the train dataset.
train = preProcessData(train)
valid = preProcessData(valid)
# test = preProcessData(test)

In [5]:
# train.head()

In [6]:
# valid.head(3)

In [7]:
# test.head(3)

### Lets keep only relevant records in Valid and test set.

In [9]:
abbrev = list(train['ABV'].unique())
valid = valid[valid['ABV'].isin(abbrev)]
# test = test[test['ABV'].isin(abbrev)]
labels = list(train['LABEL'].unique())
valid = valid[valid['LABEL'].isin(labels)]
# test = test[test['LABEL'].isin(labels)]

# Lets tag every Token List with its Label

train_tagged = train.apply(lambda x: TaggedDocument(words = x['TOKEN'], tags = [x['LABEL']]), axis=1)
valid_tagged = valid.apply(lambda x: TaggedDocument(words = x['TOKEN'], tags = [x['LABEL']]), axis=1)
# test_tagged = test.apply(lambda x: TaggedDocument(words = x['TOKEN'], tags = [x['LABEL']]), axis=1)

# # Convert TOKEN column from string to list
# train['TOKEN'] = train['TOKEN'].apply(lambda x: ast.literal_eval(x))
# valid['TOKEN'] = valid['TOKEN'].apply(lambda x: ast.literal_eval(x))
# test['TOKEN'] = test['TOKEN'].apply(lambda x: ast.literal_eval(x))

In [10]:
train_tagged.values[:2]

array([TaggedDocument(words=['yearold', 'girl', 'presented', 'unilateral', 'c6', 'nerve', 'palsy', 'papilledema', 't3', 'severe', 'ha', 'magnetic', 'resonance', 'imaging', 'showed', 'thrombosis', 'superior', 'sagittal', 'sinus', 'pt', 'right', 'transverse', 'sinus', 'attributed', 'oral', 'contraceptive', 'use', 't3', 'coagulation', 'workup', 'negative', 'svr', 'anticoagulation', 'caused', 'hemorrhagic', 'papillopathy', 'eyes', 'raising', 'question', 'ac', 'discontinued', 'diagnostic', 'management', 'issues', 'cbf', 'venous', 'sinus', 'thrombosis', 'secondary', 'intracranial', 'hypertension', 'discussed'], tags=['anticoagulation']),
       TaggedDocument(words=['mvr', 'procedure', 'choice', 'treat', 'mitral', 'valve', 'dysfunction', 'advantages', 'mvr', 'mitral', 'vr', 'include', 'improved', 'lts', 'better', 'preservation', 'left', 'vvi', 'cf', 'greater', 'freedom', 'endocarditis', 'thromboembolism', 'anticoagulantrelated', 'hemorrhage', 'feasibility', 'durability', 'mvr', 'depend', 'et

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# Instantiate the LabelEncoder
label_encoder = LabelEncoder()

def prepare_text_and_labels(dataset, include_labels=True):
    """
    Prepares text variables and labels for abbreviation expansion tasks.
    
    Args:
        dataset (pd.DataFrame): The dataset containing 'TOKEN' and optionally 'LABEL' columns.
        include_labels (bool): Whether to return the labels along with the texts.
        
    Returns:
        list: A list of input texts.
        list (optional): A list of labels if include_labels is True.
    """
    # Generate input texts from the 'TOKEN' column
    texts = [" ".join(tokens) for tokens in dataset["TOKEN"]]
    
    # If labels are needed, extract them
    if include_labels and "LABEL" in dataset.columns:
        labels = dataset["LABEL"].tolist()
        return texts, labels
    
    return texts

# Assuming `train`, `valid`, and `test` are pandas DataFrames from your notebook

# For training set
train_texts, train_labels = prepare_text_and_labels(train)

# For validation set
valid_texts, valid_labels = prepare_text_and_labels(valid)

# For test set, if you don't need labels
# test_texts = prepare_text_and_labels(test, include_labels=False)

# Display examples
print(train_texts[:2])  # Example texts
print(train_labels[:2])  # Corresponding labels

# Fit the LabelEncoder on the training labels
label_encoder.fit(train_labels)

# Transform the training labels (if needed)
train_labels_encoded = label_encoder.transform(train_labels)

# Filter validation set to remove unseen labels
valid_filtered = valid[valid["LABEL"].isin(label_encoder.classes_)]
valid_texts, valid_labels = prepare_text_and_labels(valid_filtered)

# Transform the validation labels
valid_labels_encoded = label_encoder.transform(valid_labels)

['yearold girl presented unilateral c6 nerve palsy papilledema t3 severe ha magnetic resonance imaging showed thrombosis superior sagittal sinus pt right transverse sinus attributed oral contraceptive use t3 coagulation workup negative svr anticoagulation caused hemorrhagic papillopathy eyes raising question ac discontinued diagnostic management issues cbf venous sinus thrombosis secondary intracranial hypertension discussed', 'mvr procedure choice treat mitral valve dysfunction advantages mvr mitral vr include improved lts better preservation left vvi cf greater freedom endocarditis thromboembolism anticoagulantrelated hemorrhage feasibility durability mvr depend etiology mitral valve dysfunction degenerative icm mitral valve diseases valve repair possible cases freedom reoperation high contrast rheumatic valves amenable repair durability limited isolated valve operations performed minimally invasive fashion partial upper sternotomy patients atrial fibrillation mvd maze procedure pulm

In [30]:
# # from transformers import AutoTokenizer
# # import torch
# # from transformers import AutoModelForSequenceClassification
# # from transformers import TrainingArguments
# # from transformers import Trainer


# # # Load tokenizer
# # tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# # # Tokenize training and validation datasets
# # train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
# # valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=128)


# # class AbbreviationDataset(torch.utils.data.Dataset):
# #     def __init__(self, encodings, labels=None):
# #         self.encodings = encodings
# #         self.labels = labels

# #     def __len__(self):
# #         return len(self.encodings["input_ids"])

# #     def __getitem__(self, idx):
# #         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
# #         if self.labels is not None:
# #             item["labels"] = torch.tensor(self.labels[idx])
# #         return item

# # # Create datasets
# # train_dataset = AbbreviationDataset(train_encodings, train_labels_encoded)
# # valid_dataset = AbbreviationDataset(valid_encodings, valid_labels_encoded)



# # # Load pretrained model
# # num_labels = len(label_encoder.classes_)  # Number of unique labels
# # model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)



# # training_args = TrainingArguments(
# #     output_dir="./results",           # Directory to save model checkpoints
# #     evaluation_strategy="epoch",     # Evaluate after each epoch
# #     learning_rate=2e-5,              # Learning rate
# #     per_device_train_batch_size=16,  # Batch size for training
# #     per_device_eval_batch_size=16,   # Batch size for evaluation
# #     num_train_epochs=3,              # Number of epochs
# #     weight_decay=0.01,               # Weight decay for regularization
# #     save_total_limit=1,              # Save only the last checkpoint
# #     logging_dir="./logs",            # Directory for logs
# #     logging_steps=10,                # Log every 10 steps
# # )



# # trainer = Trainer(
# #     model=model,
# #     args=training_args,
# #     train_dataset=train_dataset,
# #     eval_dataset=valid_dataset,
# #     tokenizer=tokenizer,
# # )

# # # Train the model
# # trainer.train()

# # results = trainer.evaluate()
# # print("Validation Results:", results)

# # model.save_pretrained("./trained_model")
# # tokenizer.save_pretrained("./trained_model")


# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# import torch
# from sklearn.preprocessing import LabelEncoder

# def train_abbreviation_model(train_texts, train_labels, valid_texts, valid_labels, model_name="bert-base-uncased", output_dir="./trained_model", num_epochs=3, batch_size=16, learning_rate=2e-5):
#     """
#     Trains a BERT-based model for abbreviation expansion.
    
#     Args:
#         train_texts (list): List of training texts.
#         train_labels (list): List of training labels.
#         valid_texts (list): List of validation texts.
#         valid_labels (list): List of validation labels.
#         model_name (str): Pretrained model name from Hugging Face.
#         output_dir (str): Directory to save the trained model.
#         num_epochs (int): Number of epochs for training.
#         batch_size (int): Batch size for training and evaluation.
#         learning_rate (float): Learning rate for the optimizer.
    
#     Returns:
#         dict: Evaluation results from the validation dataset.
#     """
#     # Tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
    
#     # Tokenize datasets
#     train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
#     valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=128)
    
#     # Encode labels
#     label_encoder = LabelEncoder()
#     train_labels_encoded = label_encoder.fit_transform(train_labels)
#     valid_labels_encoded = label_encoder.transform(valid_labels)
    
#     # Create PyTorch datasets
#     class AbbreviationDataset(torch.utils.data.Dataset):
#         def __init__(self, encodings, labels=None):
#             self.encodings = encodings
#             self.labels = labels

#         def __len__(self):
#             return len(self.encodings["input_ids"])

#         def __getitem__(self, idx):
#             item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#             if self.labels is not None:
#                 item["labels"] = torch.tensor(self.labels[idx])
#             return item

#     train_dataset = AbbreviationDataset(train_encodings, train_labels_encoded)
#     valid_dataset = AbbreviationDataset(valid_encodings, valid_labels_encoded)
    
#     Load pretrained model
#     num_labels = len(label_encoder.classes_)
#     model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    
#     # Training arguments
#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         evaluation_strategy="epoch",
#         learning_rate=learning_rate,
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size,
#         num_train_epochs=num_epochs,
#         weight_decay=0.01,
#         save_total_limit=1,
#         logging_dir=f"{output_dir}/logs",
#         logging_steps=10,
#     )
    
#     # Trainer
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=valid_dataset,
#         tokenizer=tokenizer,
#     )
    
#     # Train the model
#     trainer.train()
    
#     # Evaluate the model
#     results = trainer.evaluate()
#     # print("Validation Results:", results)
    
#     # Save the model and tokenizer
#     model.save_pretrained(output_dir)
#     tokenizer.save_pretrained(output_dir)
#     print(f"Model and tokenizer saved to {output_dir}")
    
#     return results





In [31]:
# train_results, trainer, model, training_args, learning_rate = train_abbreviation_model(
#     train_texts=train_texts,
#     train_labels=train_labels,
#     valid_texts=valid_texts,
#     valid_labels=valid_labels,
#     model_name="bert-base-uncased",
#     output_dir="./trained_model",
#     num_epochs=3,
#     batch_size=16,
#     learning_rate=2e-5,
# )

# print("Training completed. Validation results:")
# print(train_results)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# import torch
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report
# import numpy as np

def train_and_evaluate_abbreviation_model(train_texts, train_labels, valid_filtered, model_name="bert-base-uncased", output_dir="./trained_model", num_epochs=3, batch_size=16, learning_rate=2e-5):
    """
    Trains a BERT-based model for abbreviation expansion and evaluates on a filtered validation dataset.
    
    Args:
        train_texts (list): List of training texts.
        train_labels (list): List of training labels.
        valid_filtered (pd.DataFrame): Filtered validation dataset with 'TOKEN' and 'LABEL' columns.
        model_name (str): Pretrained model name from Hugging Face.
        output_dir (str): Directory to save the trained model.
        num_epochs (int): Number of epochs for training.
        batch_size (int): Batch size for training and evaluation.
        learning_rate (float): Learning rate for the optimizer.
    
    Returns:
        dict: Evaluation metrics including precision, recall, F1-score, and accuracy.
    """
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Tokenize training data
    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
    
    # Encode labels for training
    label_encoder = LabelEncoder()
    train_labels_encoded = label_encoder.fit_transform(train_labels)
    
    # Prepare validation texts and labels
    valid_texts, valid_labels = prepare_text_and_labels(valid_filtered)
    
    # Tokenize validation data
    valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=128)
    valid_labels_encoded = label_encoder.transform(valid_labels)
    
    # Create PyTorch datasets
    class AbbreviationDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels=None):
            self.encodings = encodings
            self.labels = labels

        def __len__(self):
            return len(self.encodings["input_ids"])

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            if self.labels is not None:
                item["labels"] = torch.tensor(self.labels[idx])
            return item

    train_dataset = AbbreviationDataset(train_encodings, train_labels_encoded)
    valid_dataset = AbbreviationDataset(valid_encodings, valid_labels_encoded)
    
    # Load pretrained model
    num_labels = len(label_encoder.classes_)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        save_total_limit=1,
        logging_dir=f"{output_dir}/logs",
        logging_steps=10,
    )
    
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        tokenizer=tokenizer,
    )
    
    # Train the model
    trainer.train()
    
    # Evaluate the model on the validation set
    valid_preds = trainer.predict(valid_dataset)
    
    # Extract true and predicted labels
    predicted_labels = torch.argmax(torch.tensor(valid_preds.predictions), axis=1).numpy()
    true_labels = valid_labels_encoded
    
    # Compute evaluation metrics
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    accuracy = accuracy_score(true_labels, predicted_labels)
    
    # Confusion matrix and classification report
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    class_report = classification_report(true_labels, predicted_labels, target_names=label_encoder.classes_)
    
    # Save the model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Model and tokenizer saved to {output_dir}")
    
    # Return metrics and other results
    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix,
        "classification_report": class_report
    }
    
    return metrics



metrics = train_and_evaluate_abbreviation_model(
    train_texts=train_texts,
    train_labels=train_labels,
    valid_filtered=valid_filtered,  # Validation data filtered for unseen labels
    model_name="bert-base-uncased",
    output_dir="./trained_model",
    num_epochs=3,
    batch_size=16,
    learning_rate=2e-5,
)

# Display metrics
print("Evaluation Metrics:")
for key, value in metrics.items():
    if key == "confusion_matrix":
        print(f"{key}:\n{value}")
    else:
        print(f"{key}: {value}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,2.785100,2.471378,68.589400,255.520000
2,1.664100,1.628383,68.625500,255.386000
3,1.445900,1.454005,68.616400,255.420000


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: 

Model and tokenizer saved to ./trained_model
Evaluation Metrics:
accuracy: 0.697763323062878
precision: 0.673149790509698
recall: 0.697763323062878
f1_score: 0.6734324139880631
confusion_matrix:
[[ 5  0  0 ...  0  3  0]
 [ 0 80  0 ...  0  0  0]
 [ 0  0 64 ...  1  0  0]
 ...
 [ 0  0  0 ... 69  2  0]
 [ 0  0  0 ...  0 57  0]
 [ 0  0  0 ...  0  0  0]]
classification_report:                                                    precision    recall  f1-score   support

                                  aberrant crypts       1.00      0.17      0.29        29
                                 absence epilepsy       0.95      0.90      0.92        89
                                   absolute error       0.67      0.72      0.69        89
                                       acebutolol       0.91      0.92      0.92        89
                                          acetone       0.57      0.75      0.65        89
                                  acid ceramidase       0.83      0.85      0.8